In [1]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import requests
import numpy as np
from pandas.io.json import json_normalize
import requests                  
from bs4 import BeautifulSoup    
import re                        
from datetime import datetime    
from time import mktime          

In [2]:
##variables
url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/market/get-summary"
token = "22990f9b4amsh4df04e987b36e1bp18331cjsn862ed02f8065"
day_begin = "01-01-2010"
day_end = "01-01-2019"
interval = "1d"

In [3]:
#START THE PROCESS
def start():
    Stockslist = list()
    Stocknames = list()
    End_of_Party = 0
    while End_of_Party == 0:
        input_stock = (input("Insert the Name of the Company to add stock, otherwise enter \"done\": "))
        input_stock = input_stock.upper()
        if input_stock == "DONE":
            End_of_Party += 1
        else:
            Stocknames.append(input_stock)
            input_stock = (input("Insert the SHORT NAME (Stockname) to add stock, otherwise enter \"done\": "))
            input_stock = input_stock.upper()
            if input_stock == "DONE":
                End_of_Party += 1
            else:
                Stockslist.append(input_stock)
            continue
        return Stocknames, Stockslist

In [4]:
#activate the load data function
def get_data_rollin(Stocknames, Shortnames):
    index = 0
    oop = {}
    for names in Shortnames:
        print(names)
        Stockname = Stocknames[index]
        index +=1
        oop[Stockname] = get_info(names)
    return oop
    
    
def get_info(Stock):
    Stock = load_csv_data(Stock, interval, day_begin, day_end)
    return Stock

In [5]:
def _get_crumbs_and_cookies(stock):
    
    url = 'https://finance.yahoo.com/quote/{}/history'.format(stock)
    with requests.session():
        header = {'Connection': 'keep-alive',
                   'Expires': '-1',
                   'Upgrade-Insecure-Requests': '1',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) \
                   AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'
                   }
        
        website = requests.get(url, headers=header)
        soup = BeautifulSoup(website.text, 'lxml')
        crumb = re.findall('"CrumbStore":{"crumb":"(.+?)"}', str(soup))

        return (header, crumb[0], website.cookies)

In [6]:
def convert_to_unix(date):
    
    datum = datetime.strptime(date, '%d-%m-%Y')
    
    return int(mktime(datum.timetuple()))


In [7]:
def load_csv_data(stock, interval, day_begin, day_end):
  
    day_begin_unix = convert_to_unix(day_begin)
    day_end_unix = convert_to_unix(day_end)
    
    header, crumb, cookies = _get_crumbs_and_cookies(stock)
    
    with requests.session():
        url = 'https://query1.finance.yahoo.com/v7/finance/download/' \
              '{stock}?period1={day_begin}&period2={day_end}&interval={interval}&events=history&crumb={crumb}' \
              .format(stock=stock, day_begin=day_begin_unix, day_end=day_end_unix, interval=interval, crumb=crumb)
                
        website = requests.get(url, headers=header, cookies=cookies)
       
        return website.text.split('\n')[:-1]
    
    

In [8]:
def next_step(List_of_raws):
    List_col = ["Date","Open","High","Low","Close","Adj Close","Volume"]
    Amazon = list()
    for data in List_of_raws:
        for col in List_col:
            data.split(",")
            print("We've just received a shit ton of data from",data)


In [ ]:
#Separate the lists by commas
List_col = ["Date","Open","High","Low","Close","Adj Close","Volume"]
Amazon = list()
for day in Amazon_raw:
    day = day.split(",")
    Amazon.append(day)
    
Tencent = list()
for day in Tencent_raw:
    day = day.split(",")
    Tencent.append(day)
    
Alibaba = list()
for day in Alibaba_raw:
    day = day.split(",")
    Alibaba.append(day)
    
#get Dataframes.
df_amazon = pd.DataFrame(Amazon)
df_alibaba = pd.DataFrame(Alibaba)
df_tencent = pd.DataFrame(Tencent)

#first row to columnnames.
df_tencent = df_tencent.rename(columns=df.iloc[0])
df_amazon = df_amazon.rename(columns=df.iloc[0])
df_alibaba = df_alibaba.rename(columns=df.iloc[0])

#delete 1st row becaause those are the column names now. 
df_tencent = df_tencent.drop(0,axis=0)
df_amazon = df_amazon.drop(0, axis=0)
df_alibaba = df_alibaba.drop(0, axis=0)
#function to change column dtypes 
def change_to_numeric(Columns, Dataframe):
    for col in Columns:
        Dataframe[col] = pd.to_numeric(Dataframe[col])
Col_dtype_to_float = ["Open","High","Low","Close","Adj Close","Volume"]
#execute change of dtypes from columns
change_to_numeric(Col_dtype_to_float, df_tencent)
change_to_numeric(Col_dtype_to_float, df_alibaba)
change_to_numeric(Col_dtype_to_float, df_amazon)





In [ ]:
#change of column names except of Date:
x = {}
Col_dtype_to_float = ["Open","High","Low","Close","Adj Close","Volume"]
def changecolname(old_cols, Dataframe, NameofComp):
    for col in old_cols:
        x.update({col: NameofComp+col})
    Dataframe = Dataframe.rename(columns = x)
    return Dataframe
 
df_alibaba = changecolname(Col_dtype_to_float, df_alibaba, "Alibaba")
df_amazon = changecolname(Col_dtype_to_float, df_amazon, "Amazon")
df_tencent = changecolname(Col_dtype_to_float, df_tencent, "Tencent")

In [ ]:
#merge all Dataframes: 
df_ali_ama = df_alibaba.merge(df_amazon, how="inner", on="Date")
df_all = df_ali_ama.merge(df_tencent, how="inner", on="Date")

In [ ]:
N):
    empty = list()
    for x in Dataframe[Name + "Close"]:
        if len(empty) < 0.1:
            y = x
        perc = x / y
        empty.append(perc)
    Liste = empty
    return Liste

ama = percentage(df_all, ama, "Amazon")
ali = percentage(df_all, ali, "Alibaba")
ten = percentage(df_all, ten, "Tencent")


In [ ]:
df_close_all = df_all[["AlibabaClose","AmazonClose","TencentClose"]]
plt.plot(df_close_all)

In [9]:
Startprocess = start()
List_of_raws = get_data_rollin(Startprocess[0],Startprocess[1])
df = pd.DataFrame(List_of_raws)


Insert the Name of the Company to add stock, otherwise enter "done":  AmaZon
Insert the SHORT NAME (Stockname) to add stock, otherwise enter "done":  amzn
Insert the Name of the Company to add stock, otherwise enter "done":  apple
Insert the SHORT NAME (Stockname) to add stock, otherwise enter "done":  aapl
Insert the Name of the Company to add stock, otherwise enter "done":  done


AMZN
AAPL


In [10]:
def getting_data_seperated(List_of_raws):
    list2 = list()
    for x in List_of_raws.values():
        temp = list()
        for z in x:
            temp.append(z.split(","))
        list2.append(temp)
    list2 = creating_dfs(Startprocess[0], list2)
    return list2
#df2 = pd.DataFrame(list2)
#df2.transpose()

In [41]:
"""
def creating_dfs(Compnames, List):
    
    df_all = pd.DataFrame(List[0]).transpose()
    for i in range(1,len(Compnames)):
        df = pd.DataFrame(List[i]).transpose()
         
        df_all = df.merge(df_all, on="index")            
    return df_all
"""

'\ndef creating_dfs(Compnames, List):\n    \n    df_all = pd.DataFrame(List[0]).transpose()\n    for i in range(1,len(Compnames)):\n        df = pd.DataFrame(List[i]).transpose()\n         \n        df_all = df.merge(df_all, on="index")            \n    return df_all\n'

In [57]:

def creating_dfs(Compnames, List):
    df_all = pd.DataFrame(List[0])
    df_all = df_all.rename(columns=df_all.iloc[0])
    df_all = df_all.drop(0,axis=0)

    for i in range((len(Compnames)-1)):
        df = pd.DataFrame(List[i+1]) 
        df = df.rename(columns=df.iloc[0])
        df = df.drop(0,axis=0)
        df_all = pd.merge(df_all,df, on="Date") 
        
        
    return df_all

In [58]:
df_all = getting_data_seperated(List_of_raws)
#df_all = pd.DataFrame(df_all)
#df_all.transpose()
#Startprocess[0]
#List_of_raws.values()
df_all

(2265, 7)
            Date         Open         High          Low        Close  \
1     2009-12-31   137.089996   137.279999   134.520004   134.520004   
2     2010-01-04   136.250000   136.610001   133.139999   133.899994   
3     2010-01-05   133.429993   135.479996   131.809998   134.690002   
4     2010-01-06   134.600006   134.729996   131.649994   132.250000   
5     2010-01-07   132.009995   132.320007   128.800003   130.000000   
6     2010-01-08   130.559998   133.679993   129.029999   133.520004   
7     2010-01-11   132.619995   132.800003   129.210007   130.309998   
8     2010-01-12   128.990005   129.820007   126.550003   127.349998   
9     2010-01-13   127.900002   129.710007   125.750000   129.110001   
10    2010-01-14   129.139999   130.380005   126.400002   127.349998   
11    2010-01-15   129.179993   129.649994   127.059998   127.139999   
12    2010-01-19   126.309998   128.000000   124.330002   127.610001   
13    2010-01-20   127.129997   129.199997   125.08000

,Date,Open_x,High_x,Low_x,Close_x,Adj Close_x,Volume_x,Open_y,High_y,Low_y,Close_y,Adj Close_y,Volume_y
0,2009-12-31,137.089996,137.279999,134.520004,134.520004,134.520004,4523000,30.447144,30.478571,30.080000,30.104286,19.996618,88102700
1,2010-01-04,136.250000,136.610001,133.139999,133.899994,133.899994,7599900,30.490000,30.642857,30.340000,30.572857,20.307869,123432400
2,2010-01-05,133.429993,135.479996,131.809998,134.690002,134.690002,8851900,30.657143,30.798571,30.464285,30.625713,20.342979,150476200
3,2010-01-06,134.600006,134.729996,131.649994,132.250000,132.250000,7178800,30.625713,30.747143,30.107143,30.138571,20.019398,138040000
4,2010-01-07,132.009995,132.320007,128.800003,130.000000,130.000000,11030200,30.250000,30.285715,29.864286,30.082857,19.982389,119282800
5,2010-01-08,130.559998,133.679993,129.029999,133.520004,133.520004,9830500,30.042856,30.285715,29.865715,30.282858,20.115240,111902700
6,2010-01-11,132.619995,132.800003,129.210007,130.309998,130.309998,8779400,30.400000,30.428572,29.778572,30.015715,19.937790,115557400
7,2010-01-12,128.990005,129.820007,126.550003,127.349998,127.349998,9096300,29.884285,29.967142,29.488571,29.674286,19.710993,148614900
8,2010-01-13,127.900002,129.710007,125.750000,129.110001,129.110001,10723200,29.695715,30.132856,29.157143,30.092857,19.989031,151473000
9,2010-01-14,129.139999,130.380005,126.400002,127.349998,127.349998,9774900,30.015715,30.065714,29.860001,29.918571,19.873259,108223500


In [ ]:
#df2 = pd.DataFrame(list2[0])
#df3 = pd.DataFrame(list2[1])